In [62]:
import pathlib
import os
import random
import time
import timeit
import functools
from collections import OrderedDict
import requests
import sys

from time import sleep

# Everything is an object, so is a function

In [2]:
def func(x='abc', y=42, z=None):
    print(x, y, z)

func1 = func
type(func1)
func1()

abc 42 None


In [3]:
print(id(func1), id(func))
func1()

4525258304 4525258304
abc 42 None


#### Memory id for variables

In [5]:
a = 50 
b = a
c = b
a = 3
print(c)
print(id(a))
print(id(b))
print(id(c))

50
4471423280
4471424784
4471424784


#### Functions!!!

In [9]:
def func(f):
    f()

def foo():
    print('Hello from foo')

def bar():
    print('Hello from bar')

func(foo)
func(bar)

Hello from foo
Hello from bar
Hello from foo


In [15]:
def foo():
    print('Hello from foo')

# transparent 
def func(f):
    print("Returning func f")
    return f

result = func(foo)
print(type(result))
foo()
result()

Returning func f
<class 'function'>
Hello from foo
Hello from foo


In [19]:
def hello_factory():
    def internal():
        print('Hello from internal')
    return internal

func = hello_factory()
func()
# internal()

Hello from internal


In [30]:
def transparent_factory(f):
    def internal():
        f()       
    return internal

foo_transparent = transparent_factory(foo)
foo_transparent()
print(foo_transparent.__name__)
foo()

Hello from foo
internal
Hello from foo


In [52]:
def hello_factory_2(f):
    def internal():
        print(f">>> Is about to call '{f.__name__}'\n")
        f()
        print(f"\n>>> Just called '{f.__name__}'")
        
    return internal

foo_verbosed = hello_factory_2(foo)
print(foo_verbosed.__name__)
#internal()
foo_verbosed()

#foo()

internal
>>> Is about to call 'foo'

Hello from foo

>>> Just called 'foo'


In [69]:
def foo():
    time.sleep(2)
    print('Hello from foo')

def profile(f):
    def internal():
        start = time.time()
        f()
        finish = time.time()
        print(f'Elapsed time: {finish - start}s')
    return internal

foo_profiled = profile(foo)
foo_profiled()

foo()

Hello from foo
Elapsed time: 2.0001649856567383s
Hello from foo


# Decorators

####  D. is a function that adds some additional functionality, typically not changing original behaviour. 

<img src=https://d33wubrfki0l68.cloudfront.net/12c8a296cc396d418b5407a4a4c6f9fd7d85f597/e8a54/wp-content/uploads/2018/06/python-decorator.png>

In [79]:
def profile(f):
    def internal():
        start = time.time()
        f()
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

foo_profiled = profile(foo)
foo_profiled()

Hello from foo
Elapsed time (foo): 2.002146005630493s


In [103]:
def fetch_url():
    res = requests.get('https://google.com')
    print('Content: \t', res.content[:100])

In [104]:
fetch_url_profiled = profile(fetch_url)
fetch_url_profiled()

Content: 	 b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'
Elapsed time (fetch_url): 0.31726694107055664s


####  We measure the time to get the first 100 characters on different sites

In [108]:
def fetch_url(url):
    res = requests.get(url)
    print(f'\nContent for "{url}":\t ', res.content[:100])

def profile(f):
    def internal(url):
        start = time.time()
        f(url)
        print(f'\nElapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url_profiled = profile(fetch_url)
fetch_url_profiled('https://google.com')
fetch_url_profiled('https://ithillel.ua')
fetch_url_profiled('https://lms.ithillel.ua')


Content for "https://google.com":	  b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'

Elapsed time (fetch_url): 0.36908483505249023s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="uk">\n\t<h'

Elapsed time (fetch_url): 0.20659780502319336s

Content for "https://lms.ithillel.ua":	  b'<!DOCTYPE html><html lang="uk"><head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="'

Elapsed time (fetch_url): 0.18631196022033691s


In [109]:
def profile(f):
    def internal(*args):
        start = time.time()
        f(*args)
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url = profile(fetch_url)
fetch_url('https://ithillel.ua')


Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="uk">\n\t<h'
Elapsed time (fetch_url): 0.2108161449432373s


In [124]:
def fetch_url(url, first_n=None):
    res = requests.get(url)
    print(f'\nContent for "{url}":\t ', res.content[:first_n] if first_n else res.content)

def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        f(*args, **kwargs)
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url = profile(fetch_url)
#fetch_url('https://ithillel.ua')
fetch_url('https://ithillel.ua', first_n=42)
fetch_url('https://ithillel.ua', first_n=100)
fetch_url('https://ithillel.ua', first_n=1024)


Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        '
Elapsed time (fetch_url): 0.19959020614624023s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="uk">\n\t<h'
Elapsed time (fetch_url): 0.21468114852905273s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="uk">\n\t<head>\n\n\t\t<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_normal.woff2" as="font" type="font/woff2" crossorigin="anonymous">\n<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_semibold.woff2" as="font" type="font/woff2" crossorigin="anonymous">\n<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_bold.woff2" as="font" type="

In [125]:
def fetch_url(url, first_n=100):
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url = profile(fetch_url)
with open('content.html', 'w+') as f:
    f.write(str(fetch_url('https://ithillel.ua', first_n=42)))

# cat /tmp/content.html


Elapsed time (fetch_url): 0.23211216926574707s


In [127]:
def fetch_url(url, first_n=100):
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content 

def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal

fetch_url = profile(fetch_url)
with open('content_1.html', 'w+') as f:
    f.write(str(fetch_url('https://ithillel.ua', first_n=42)))

Elapsed time for function fetch_url with params ('https://ithillel.ua',), {'first_n': 42}: 0.2088780403137207ms


####  Syntactic sugar

In [128]:
@profile # -> fetch_url = profile(fetch_url)
def fetch_url(url, first_n=100):
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content
    
print(fetch_url('https://ithillel.ua', first_n=42))
print(fetch_url('https://google.com'))

Elapsed time for function fetch_url with params ('https://ithillel.ua',), {'first_n': 42}: 0.24674105644226074ms
b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        '
Elapsed time for function fetch_url with params ('https://google.com',), {}: 0.3575930595397949ms
b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'


####  doc_string

In [129]:
def test(a, b):
    """ This function is make sum of two int """
    return a+b

help(test)

Help on function test in module __main__:

test(a, b)
    This function is make sum of two int



In [137]:
def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal


@profile
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

help(fetch_url)

Help on function internal in module __main__:

internal(*args, **kwargs)



In [138]:
def profile(f):
    @functools.wraps(f)
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal


@profile
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content


help(fetch_url)

Help on function fetch_url in module __main__:

fetch_url(url, first_n=100)
    Fetch a given url



In [139]:
def profile(msg='Elapsed time'):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s')
            return result
        return deco
    return internal

In [144]:
@profile(msg='час пройшов')
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url('https://google.com')
    
@profile(msg='Elapsed time')
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url('https://google.com')

час пройшов (fetch_url): 0.35432910919189453s
Elapsed time (fetch_url): 0.34512782096862793s


b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'

In [145]:
@profile('Elapsed time') # -> profile_ = profile('Time spent')
                         # -> foo5 = profile_(foo5)
def foo():
    """Help for foo"""
    return 42

help(foo)
print("RESULT: ", foo())

Help on function foo in module __main__:

foo()
    Help for foo

Elapsed time (foo): 9.5367431640625e-07s
RESULT:  42
